In [ ]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm

In [ ]:
%%time
pd_train = pd.read_csv("/content/drive/MyDrive/Human Activity Recognition/train.csv")
pd_test = pd.read_csv("/content/drive/MyDrive/Human Activity Recognition/train.csv")

In [ ]:
pd_train.head()

In [ ]:
pd_test.head()

In [ ]:
%%time
pd_data  = pd.concat([pd_train , pd_test])
print(pd_data.head())
pd_data.reset_index(inplace = True)
pd_data.drop('index' , inplace = True, axis = 1)

In [ ]:
pd_data.head()

In [ ]:
pd_data.shape

In [ ]:
print(pd_data.columns)

In [ ]:
y = pd_data['Activity'].copy()
x = pd_data.iloc[:,:-1]

In [ ]:
x.head()

In [ ]:
columnNames = list()
columnNames = pd_data.columns
columnNames

In [ ]:
list(columnNames)

In [ ]:
bad_char = ['-','()','(',')',',']

In [ ]:
newcolumnNames = []
for colname in columnNames:
  newcolumnName = ''.join(i for i in colname if i not in bad_char)
  newcolumnNames.append(newcolumnName)

In [ ]:
list(newcolumnNames)

In [ ]:

pd_data.columns = newcolumnNames

In [ ]:
pd_data.head()

In [ ]:
pd_data.isnull().sum()

In [ ]:

%%time
y.value_counts().plot.bar()

In [ ]:
%%time
pd_data.groupby('Activity').mean()['tBodyAccmeanX']

In [ ]:
%%time
pd_data.groupby('Activity').mean()['tBodyAccstdX']

In [ ]:
%%time
sns.set_style('whitegrid')
plt.rcParams['font.family'] = 'Dejavu Sans'
plt.figure(figsize=(16,10))
plt.title("Data collected per person",fontsize = 15)
sns.countplot(x = 'subject' , hue = 'Activity', data = pd_data)
plt.show()

In [ ]:
%%time
facetgrid = sns.FacetGrid(pd_data, hue = 'Activity', size = 6 , aspect =2 )
facetgrid.map(sns.distplot,'tBodyAccMagmean', hist = False).add_legend()
plt.annotate("Stationary Activities", xy = (-0.956, 15) , xytext = (-0.9,15), size = 15, \
             va = 'center', ha = 'left', \
             arrowprops = dict(arrowstyle ='simple', connectionstyle = 'arc3,rad = 0.1'))
plt.annotate("Moving Activities", xy = (0.0, 3) , xytext = (0.0,5), size = 15, \
             va = 'center', ha = 'left', \
             arrowprops = dict(arrowstyle ='simple', connectionstyle = 'arc3,rad = 0.1'))

In [ ]:
%%time
facetgrid = sns.FacetGrid(pd_data, hue = 'Activity', size = 6 , aspect =2 )
facetgrid.map(sns.distplot,'fBodyAccMagmean', hist = False).add_legend()
plt.annotate("Stationary Activities", xy = (-0.956, 15) , xytext = (-0.9,15), size = 15, \
             va = 'center', ha = 'left', \
             arrowprops = dict(arrowstyle ='simple', connectionstyle = 'arc3,rad = 0.1'))
plt.annotate("Moving Activities", xy = (0.0, 3) , xytext = (0.0,5), size = 15, \
             va = 'center', ha = 'left', \
             arrowprops = dict(arrowstyle ='simple', connectionstyle = 'arc3,rad = 0.1'))

In [ ]:
%%time
facetgrid = sns.FacetGrid(pd_data, hue = 'Activity', size = 6 , aspect =2 )
facetgrid.map(sns.distplot,'tBodyAccJerksma', hist = False).add_legend()
plt.annotate("Stationary Activities", xy = (-0.956, 15) , xytext = (-0.9,15), size = 15, \
             va = 'center', ha = 'left', \
             arrowprops = dict(arrowstyle ='simple', connectionstyle = 'arc3,rad = 0.1'))
plt.annotate("Moving Activities", xy = (0.0, 3) , xytext = (0.0,5), size = 15, \
             va = 'center', ha = 'left', \
             arrowprops = dict(arrowstyle ='simple', connectionstyle = 'arc3,rad = 0.1'))

In [ ]:
%%time
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
y = y.map({'STANDING': 0, 'SITTING': 1, 'LAYING': 2, 'WALKING' : 3, 'WALKING_DOWNSTAIRS': 4, 'WALKING_UPSTAIRS': 5})

In [ ]:
y.unique()

In [ ]:
print(y.shape , x.shape)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
%%time
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2 , random_state = 10)

In [ ]:
print(x_train.shape, x_test.shape)

In [ ]:
%%time 
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression(penalty = 'l2', solver = 'liblinear')

In [ ]:
%%time
logreg.fit(x_train,y_train)

In [ ]:

%%time 
y_pred = logreg.predict(x_test)

In [ ]:
y_pred[0:10]

In [ ]:
y_test[0:10]

In [ ]:

%%time
accuracy_score(y_test,y_pred)

In [ ]:
%%time
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
parameters = {
    'kernel' : ['linear','rbf'],
    'degree' : [2,3]

}

In [ ]:
%%time
svclf = SVC(kernel= 'linear', gamma = 0.1)
GridSVC = GridSearchCV(svclf, param_grid= parameters, n_jobs= -1)
svclf.fit(x_train,y_train)

In [ ]:
bestsvc=GridSVC.set_params

In [ ]:
bestsvc().fit(x_train,y_train)

In [ ]:
import cudf as gdf

In [ ]:
%%time
gpu_train = gdf.read_csv('/content/drive/MyDrive/Human Activity Recognition/train.csv')
gpu_test = gdf.read_csv('/content/drive/MyDrive/Human Activity Recognition/test.csv')

In [ ]:
!nvidia-smi

In [ ]:
from cuml.model_selection import train_test_split

In [ ]:
%%time
gpu_data = gdf.concat([gpu_train,gpu_test])
gpu_data.reset_index(inplace = True)
gpu_data.drop('index',inplace = True , axis = 1)

In [ ]:
!nvidia-smi

In [ ]:
%%time

gpu_data.groupby('Activity').mean()

In [ ]:
from cuml.preprocessing import LabelEncoder

In [ ]:
le  = LabelEncoder()

In [ ]:
%%time
x_gpu = gpu_data.drop('Activity', inplace=False, axis=1)
y_gpu = gpu_data['Activity'].copy()

In [ ]:
%%time
le.fit(y_gpu)
encodedLabels = le.transform(y_gpu)

In [ ]:
y_gpu = y_gpu.map({'STANDING':0,'SITTING':1,'LAYING':2,'WALKING':3,'WALKING_DOWNSTAIRS':4, 'WALKING_UPSTAIRS':5})

In [ ]:
x_gpu = x_gpu.drop('subject',inplace=False, axis = 1)

In [ ]:
%%time 
x_train_gpu,x_test_gpu,y_train_gpu,y_test_gpu = train_test_split(x_gpu, y_gpu , test_size  = 0.2 , random_state = 12)

In [ ]:
import cupy as cup # here this is a replacement for numpy

In [ ]:
cup.random.seed(10)
y_pred_gpu = cup.random.randint(low = 0 , high = 8)

In [ ]:
from cuml.linear_model import LogisticRegression

In [ ]:
%%time
gpu_log = LogisticRegression(solver='qn')

In [ ]:
%%time
gpu_log.fit(x_train_gpu,y_train_gpu)

In [ ]:
%%time
from cuml.svm import SVC 
from cuml.model_selection import GridSearchCV

In [ ]:
%%time
parameters = {
    'kernel' : ['linear','rbf'],
    'degree' : [2,3]

}

In [ ]:
%%time
svc_gpu  = SVC(kernel= 'linear',gamma=0.1)

In [ ]:
%%time
svc_gpu.fit(x_train_gpu,y_train_gpu)

In [ ]:
%%time
y_pred_gpu = svc_gpu.predict(x_test_gpu)

In [ ]:
%%time
y_pred_gpu_1 = gpu_log.predict(x_test_gpu)

In [ ]:
from cuml.metrics import confusion_matrix, accuracy_score

In [ ]:
%%time
confusion_matrix(y_test_gpu,y_pred_gpu)

In [ ]:
%%time
confusion_matrix(y_test_gpu,y_pred_gpu_1)

In [ ]:
%%time
print(accuracy_score(y_test_gpu,y_pred_gpu))
print(accuracy_score(y_test_gpu,y_pred_gpu_1))